In [65]:
using CSV
using DataFrames

## 2次元情報の読み込み  
geometry_data内にはそれぞれ下記の情報が記されている。
- 各位置の1次元の壁情報
- CVの大きさ（縦・横）
- 境界条件の情報

A行および1列目は各CVの幅  
B行および2列目と各最終行・列は各CVの境界条件  
C行および3列目からは各CVがどの壁情報を対応するのか  
である。

In [2]:
function read_geometry_data(file_name::String)

    # 2次元情報の読み込み
    geometry_data = CSV.File(file_name, header = false)

    # y軸およびz軸のCVの数の読み取り
    num_y = length(geometry_data[1]) - 3
    num_z = length(geometry_data) - 3

    # y軸およびz軸のCVの幅の読み取り
    dy = [ geometry_data[1][y] for y = 3 : num_y + 2 ]
    dz = [ geometry_data[z][1] for z = 3 : num_z + 2 ]

    # 各軸における境界情報の読み取り
    bound_info_y_axis_tip = [ Int(geometry_data[2][y]) for y = 3 : num_y + 2 ]
    bound_info_z_axis_tip = [ Int(geometry_data[z][2]) for z = 3 : num_z + 2 ]
    bound_info_y_axis_end = [ Int(geometry_data[end][y]) for y = 3 : num_y + 2 ]
    bound_info_z_axis_end = [ Int(geometry_data[z][end]) for z = 3 : num_z + 2 ]

    # 各点における壁番号の読み取り
    wall_number = [ Int(geometry_data[z][y]) for y = 3 : num_y + 2, z = 3 : num_z + 2 ]

    return dy, dz, bound_info_y_axis_tip, bound_info_z_axis_tip, bound_info_y_axis_end, bound_info_z_axis_end, wall_number
end 

read_geometry_data (generic function with 1 method)

In [3]:
# Cell構造体の設定
Base.@kwdef mutable struct Cell
    i::Array{Int, 1} = [ 1, 1, 1 ]     #= 位置  =#
    dx::Float64      = 1.0   #= 幅x   =#
    dy::Float64      = 1.0   #= 高さy =#
    dz::Float64      = 1.0   #= 奥行z =#
    dx2::Float64     = 1.0   # 質点からセル端までの距離（一般的にはdxの半分）
    dy2::Float64     = 1.0   
    dz2::Float64     = 1.0   
    temp::Float64    = 0.0   #= 温度 =#
    miu::Float64     = 0.0   #= 水分化学ポテンシャルの毛管圧成分（いわゆる水分化学ポテンシャル）=#
    rh::Float64      = 0.0   #= 相対湿度 =#
    pv::Float64      = 0.0   #= 水蒸気圧 =#
    phi::Float64     = 0.0   #= 含水率 =#
    material_name::String   = "NoName"  #= 材料名 =#
end 

Cell

In [10]:
function read_multi2D_geometry_data(file_number::Int, temp_init::Float64, RH_init::Float64)

    # 3次元のプロパティ情報の読み込み
    readed_property_data = [ CSV.File("./2D_property_data/"*"2D_property_data_"*string(i)*".csv", header = false) for i = 1 : file_number ]
    property_data = permutedims([ readed_property_data[x][y][z] for x = 1:length(readed_property_data), y = 1:length(readed_property_data[1]), z = 1:length(readed_property_data[1][1]) ], [1,3,2] )
    property_info = DataFrame(CSV.File("property_information.csv", header = 1))

    # 各CVの大きさ（dx, dy, dz）情報の読み込み
    dx_data = CSV.File("dx_data.csv", header = 1)
    dy_data = CSV.File("dy_data.csv", header = 1)
    dz_data = CSV.File("dz_data.csv", header = 1)
    
    # Cellおよび空間を含めた構造体の作成
    wall = [ Cell() for x = 1 : length(dx_data.i) , y = 1 : length(dy_data.i), z = 1 : length(dz_data.i) ]

    # 各点における壁番号の読み取り
    # 各CVの情報を入力する。なお、i,dx,dx2,temp,miu（水分化学ポテンシャル）,material_nameは入力必須項目である。
    for x = 1 : length(dx_data.i) , y = 1 : length(dy_data.i), z = 1 : length(dz_data.i)
        wall[x, y, z].i   = [ x, y, z ]
        wall[x, y, z].dx  = dx_data.dx[x]
        wall[x, y, z].dx2 = dx_data.dx2[x]
        wall[x, y, z].dy  = dy_data.dy[x]
        wall[x, y, z].dy2 = dy_data.dy2[x]
        wall[x, y, z].dz  = dz_data.dz[x]
        wall[x, y, z].dz2 = dz_data.dz2[x]
        wall[x, y, z].temp= temp_init
        wall[x, y, z].rh  = rh_init
        #wall[x, y, z].miu = convertRH2Miu( temp = wall[x, y, z].temp, rh = wall[x, y, z].rh )
        wall[x, y, z].material_name = filter(row -> row.number == property_data[x+1,y+1,z+1], property_info).name[1]
    end
    
    return property_data, wall
end 

read_multi2D_geometry_data (generic function with 1 method)

In [102]:
file_number = 7
temp_init   = 293.15
rh_init     = 0.6

    # 3次元のプロパティ情報の読み込み
    readed_property_data = [ CSV.File("./2D_property_data/"*"2D_property_data_"*string(i)*".csv", header = false) for i = 1 : file_number ]
    property_data = permutedims([ readed_property_data[x][y][z] for x = 1:length(readed_property_data), y = 1:length(readed_property_data[1]), z = 1:length(readed_property_data[1][1]) ], [1,3,2] )
    property_info = DataFrame(CSV.File("property_information.csv", header = 1))

    # 各CVの大きさ（dx, dy, dz）情報の読み込み
    dx_data = CSV.File("dx_data.csv", header = 1)
    dy_data = CSV.File("dy_data.csv", header = 1)
    dz_data = CSV.File("dz_data.csv", header = 1)
    
    # Cellおよび空間を含めた構造体の作成
    wall = [ Cell() for x = 1 : length(dx_data.i) , y = 1 : length(dy_data.i), z = 1 : length(dz_data.i) ]

    # 各点における壁番号の読み取り
    # 各CVの情報を入力する。なお、i,dx,dx2,temp,miu（水分化学ポテンシャル）,material_nameは入力必須項目である。
    for x = 1 : length(dx_data.i) , y = 1 : length(dy_data.i), z = 1 : length(dz_data.i)
        wall[x, y, z].i   = [ x, y, z ]
        wall[x, y, z].dx  = dx_data.dx[x]
        wall[x, y, z].dx2 = dx_data.dx2[x]
        wall[x, y, z].dy  = dy_data.dy[x]
        wall[x, y, z].dy2 = dy_data.dy2[x]
        wall[x, y, z].dz  = dz_data.dz[x]
        wall[x, y, z].dz2 = dz_data.dz2[x]
        wall[x, y, z].temp= temp_init
        wall[x, y, z].rh  = rh_init
        #wall[x, y, z].miu = convertRH2Miu( temp = wall[x, y, z].temp, rh = wall[x, y, z].rh )
        wall[x, y, z].material_name = filter(row -> row.number == property_data[x+1,y+1,z+1], property_info).name[1]

    end

In [103]:
wall

5×8×16 Array{Cell, 3}:
[:, :, 1] =
 Cell([1, 1, 1], 0.01, 0.05, 0.05, 0.0, 0.0, 0.0, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")            …  Cell([1, 8, 1], 0.01, 0.05, 0.05, 0.0, 0.0, 0.0, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")
 Cell([2, 1, 1], 0.01, 0.05, 0.05, 0.005, 0.025, 0.025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")         Cell([2, 8, 1], 0.01, 0.05, 0.05, 0.005, 0.025, 0.025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")
 Cell([3, 1, 1], 0.01, 0.05, 0.05, 0.005, 0.025, 0.025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")         Cell([3, 8, 1], 0.01, 0.05, 0.05, 0.005, 0.025, 0.025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")
 Cell([4, 1, 1], 0.01, 0.005, 0.005, 0.005, 0.0025, 0.0025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")     Cell([4, 8, 1], 0.01, 0.005, 0.005, 0.005, 0.0025, 0.0025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")
 Cell([5, 1, 1], 0.01, 0.005, 0.005, 0.01, 0.0025, 0.0025, 293.15, 0.0, 0.6, 0.0, 0.0, "mud_wall")      Cell([5, 8, 1], 0.01, 0.005, 0.005, 0.01, 0.0025, 0.0025, 293.15, 0